In [42]:
from typing import TypedDict, Annotated, List
import operator
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from pydantic import BaseModel
import os
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
model = ChatGroq(model="llama-3.3-70b-versatile")

In [44]:
class Evaluation(BaseModel):
    feedback: str
    score: int

structured_model = model.with_structured_output(Evaluation)

In [45]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list, operator.add]
    avg_score: float

In [46]:
def evaluate_language(state: UPSCState):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assess it out of 10. \n\n Essay: {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

In [47]:
def evaluate_analysis(state: UPSCState):
    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assess it out of 10. \n\n Essay: {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [48]:
def evaluate_thought(state: UPSCState):
    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assess it out of 10. \n\n Essay: {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

In [49]:
def final_evaluation(state: UPSCState):
    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback: {state["language_feedback"]} \n analysis_feedback: {state["analysis_feedback"]} \n clarity_feedback: {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content
    
    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])
    
    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [50]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [51]:
essay_1 = """
India in the Age of AI

As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture. With its vast demographic dividend, a burgeoning tech ecosystem, and a government increasingly focused on digital transformation, AI offers both immense opportunities and significant challenges for the nation.

The opportunities are manifold. In healthcare, AI-driven diagnostics can bridge the gap in rural areas where medical professionals are scarce. In agriculture, precision farming powered by AI can help Indian farmers optimize crop yields and manage water resources more efficiently. Furthermore, AI can revolutionize education through personalized learning platforms, making quality education accessible to millions.

However, the path to AI integration is fraught with challenges. The most pressing concern is the displacement of labor, particularly in sectors like IT and manufacturing. Ensuring that the workforce is reskilled to thrive in an AI-driven economy is a monumental task. Additionally, issues related to data privacy, ethical AI, and the digital divide must be addressed to ensure that the benefits of AI are equitably distributed.

In conclusion, India's journey in the age of AI will depend on its ability to harness the technology's potential while mitigating its risks. A collaborative effort between the government, industry, and academia is essential to build a robust AI ecosystem that drives inclusive growth and positions India as a global leader in the AI revolution.
"""

print("--- Evaluating Essay 1 (Good Essay) ---")
result_1 = workflow.invoke({'essay': essay_1})
import pprint
pprint.pprint(result_1)
print("\n")

--- Evaluating Essay 1 (Good Essay) ---
{'analysis_feedback': 'The essay provides a good overview of the opportunities '
                      'and challenges that India faces in the age of AI. It '
                      'highlights the potential benefits of AI in various '
                      'sectors such as healthcare, agriculture, and education, '
                      'and also discusses the challenges related to job '
                      'displacement, data privacy, and the digital divide. '
                      'However, the analysis could be deeper and more nuanced, '
                      'with more specific examples and data to support the '
                      'arguments. Additionally, the essay could benefit from a '
                      'more detailed discussion of the role of the government, '
                      'industry, and academia in building a robust AI '
                      'ecosystem. Overall, the essay provides a good '
                      'foundat

In [52]:
essay_2 = """
AI is very good and fast. India has many people and robots. 
I like technology because it is clean. Farming is slow but robots make it fast.
Schools are boring, we need AI to play games. 
Artificial intelligence is future. Total win for India. Bye.
"""

print("--- Evaluating Essay 2 (Worst Essay) ---")
result_2 = workflow.invoke({'essay': essay_2})
import pprint
pprint.pprint(result_2)

--- Evaluating Essay 2 (Worst Essay) ---
{'analysis_feedback': 'The essay lacks depth and coherence, with disjointed '
                      'sentences and simplistic language. It fails to provide '
                      'meaningful analysis or supporting evidence for the '
                      'claims made. The writing is childish and does not '
                      'demonstrate a clear understanding of the topic.',
 'avg_score': 2.0,
 'clarity_feedback': 'The essay lacks coherence and clarity of thought. The '
                     'ideas are disjointed and lack proper connection. The '
                     'language used is simple and lacks depth. The essay does '
                     'not provide any substantial argument or analysis. '
                     'However, it does show some enthusiasm for technology and '
                     'AI. Overall, the essay needs significant improvement in '
                     'terms of structure, content, and critical thinking.',
 'essay': '\